In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data_eng = pd.read_csv("English_data90k.csv")

In [4]:
data_eng.head()

,senti,text
0,1,pretty awesome place it was neat being within ...
1,0,the people who gave this place one star were r...
2,0,wow this place is pure evil i went here to att...
3,2,the tea here is delicious i got rose tea the f...
4,0,dissapointedi was waiting for the chicken frie...


In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.model_selection import train_test_split

In [12]:
import keras

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type

In [9]:
!pip install pickle


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
You should consider upgrading via the 'c:\anaconda3\python.exe -m pip install --upgrade pip' command.


In [12]:
damn = pd.read_csv("English_data90k.csv")

In [13]:
damn.head()

,senti,text
0,1,pretty awesome place it was neat being within ...
1,0,the people who gave this place one star were r...
2,0,wow this place is pure evil i went here to att...
3,2,the tea here is delicious i got rose tea the f...
4,0,dissapointedi was waiting for the chicken frie...


In [15]:
tokenizer = Tokenizer(num_words = 5000, split = " ")
tokenizer.fit_on_texts(damn["text"].values)
X = pad_sequences(tokenizer.texts_to_sequences(damn["text"]))


In [16]:
import pickle

# saving
with open('tokenizer90k.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
model = Sequential()
model.add(Embedding(5000, 256, input_length = X.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256, dropout=0.3, recurrent_dropout = 0.2))
model.add(Dense(units=3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 981, 256)          1280000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 981, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 1,806,083
Trainable params: 1,806,083
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
Y = pd.get_dummies(data_eng["senti"])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=40)

In [23]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(63000, 981) (63000, 3)
(27000, 981) (27000, 3)


In [25]:
model.fit(X_train, y_train, epochs=8, validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 56700 samples, validate on 6300 samples
Epoch 1/8
56700/56700 [==============================] - 19537s 345ms/step - loss: 0.6385 - accuracy: 0.7375 - val_loss: 0.5430 - val_accuracy: 0.7940
Epoch 2/8
56700/56700 [==============================] - 17939s 316ms/step - loss: 0.4196 - accuracy: 0.8383 - val_loss: 0.3605 - val_accuracy: 0.8627
Epoch 3/8
56700/56700 [==============================] - 10476s 185ms/step - loss: 0.3205 - accuracy: 0.8784 - val_loss: 0.3475 - val_accuracy: 0.8613
Epoch 4/8
56700/56700 [==============================] - 9814s 173ms/step - loss: 0.2733 - accuracy: 0.8956 - val_loss: 0.3285 - val_accuracy: 0.8754
Epoch 5/8
56700/56700 [==============================] - 11501s 203ms/step - loss: 0.2388 - accuracy: 0.9103 - val_loss: 0.3326 - val_accuracy: 0.8749
Epoch 6/8
56700/56700 [==============================] - 16745s 295ms/step - loss: 0.2100 - accuracy: 0.9210 - val_loss: 0.3601 - val_accuracy: 0.866

In [26]:
print("Test Accuracy")
model.evaluate(X_test, y_test)

Test Accuracy
27000/27000 [==============================] - 1230s 46ms/step


[0.42821813125522046, 0.8587407469749451]

In [27]:
print("Train Accuracy")
model.evaluate(X_train, y_train)

Train Accuracy
63000/63000 [==============================] - 2667s 42ms/step


[0.12656504158084356, 0.959507942199707]